In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

import numpy as np
from sklearn.datasets import make_classification
from sklearn import svm

In [ ]:
def evaluateBinaryClassification(predictions, actuals):
    list = []
    contigency = pd.crosstab(actuals,predictions)
    TP = contigency['yes']['yes']
    TN = contigency['no']['no']
    FP = contigency['yes']['no']
    FN = contigency['no']['yes']
    n = contigency.sum().sum()

    Acuracy = (TP + TN)/n
    Recall = TP/(TP+FN)
    Precision = TP/(TP+FP)
    FScore = 2*Recall*Precision/(Recall+Precision)
    list.append(Acuracy)
    list.append(Recall)
    list.append(Precision)
    list.append(FScore)
    # return Acuracy, Recall, Precision, FScore
    return list

In [ ]:
SEED = 40
scores = []

In [ ]:
data = pd.read_excel(r"/content/2021XRP_Full_NoScaling_Vader_Binary_Close-Open.xlsx", na_values='?',index_col=0)
print(data.dtypes)
data.head()

In [ ]:
# import collections
# # using Counter to find frequency of elements
# frequency = collections.Counter(data['S&P500_Binary'])
# print(frequency)

In [ ]:
# data = data.reset_index()
# import matplotlib.pyplot as plt
# data['percentage_change'] = ((data['BTC_Close'] - data['BTC_Open']) / data['BTC_Open']) * 100
# plt.plot(data['Date'],data['percentage_change'])
# plt.show()

In [ ]:
def yesno(x):
  if x == 1:
    return 'yes'
  else:
    return 'no'

In [ ]:
data['Ethereum_Change'] = data['Ethereum_Change'].apply(yesno)
# data['Stellar_Change'] = data['Stellar_Change'].apply(yesno)
# data['Monero_Change'] = data['Monero_Change'].apply(yesno)
# data['Litecoin_Change'] = data['Litecoin_Change'].apply(yesno)
# data['Tron_Change'] = data['Tron_Change'].apply(yesno)
# data['Cardano_Change'] = data['Cardano_Change'].apply(yesno)
# data['XRP_Change'] = data['XRP_Change'].apply(yesno)
# data['Binance_Change'] = data['Binance_Change'].apply(yesno)
# data['BTC_Change'] = data['BTC_Change'].apply(yesno)

In [ ]:
data.head()

In [ ]:
data.head()

In [ ]:
Metrics = ['Max Accuracy','Accuracy','Recall','Precision','Fscore']

compare_df = pd.DataFrame(columns = Metrics)

Decision Tree Classifier

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

  # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

  # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  classTree = DecisionTreeClassifier()
  classTree.fit(X_train, y_train)

  #plotDecisionTree(classTree, feature_names=X_train.columns, class_names=classTree.classes_)
  y_predict_DT = classTree.predict(X_test)
  # print(y_predict_DT)

  list = evaluateBinaryClassification(y_predict_DT,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]
  # compare_df.loc[len(compare_df.index)] = evaluateBinaryClassification(y_predict_DT,y_test)
  # print(compare_df)
  
avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['Decision Tree'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)


KNeighbours Classifier

In [ ]:
# x = np.arange(1,100,1)
# for k in x:
acc = 0
recall = 0
prec = 0
fscore = 0;
max_accuracy = 0;
kfold = KFold(n_splits=10, shuffle=True)
for train_index,test_index in kfold.split(data):
    # Get the train-test subsets
    X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
    y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

    # Scale the X features according to training data 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = KNeighborsClassifier(p = 1, n_neighbors=13)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    max_accuracy = max(max_accuracy,model.score(X_test, y_test))
    acc += model.score(X_test, y_test)
    recall += recall_score(y_test, y_pred, average='micro')  # average='micro' added to resolve error
    prec += precision_score(y_test, y_pred, average='micro')
    fscore += f1_score(y_test,y_pred, average='micro')
    
acc /= kfold.get_n_splits()
recall /= kfold.get_n_splits()
prec /= kfold.get_n_splits()
fscore /= kfold.get_n_splits()

compare_df.loc['KNeighbors K=13'] = [max_accuracy,acc,recall,prec,fscore]
# print(f"k={k}\tacc={acc}\trecall={recall}\tprec={prec}")
# print(compare_df)
    

Logistic Regression

In [ ]:
from sklearn import datasets, linear_model, metrics
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

               # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # create logistic regression object
  reg = linear_model.LogisticRegression(max_iter=200)
    
  # train the model using the training sets
  reg.fit(X_train, y_train)
    
  # making predictions on the testing set 
  y_predict_LR = reg.predict(X_test)
  # print(y_predict_LR)
  
  list = evaluateBinaryClassification(y_predict_LR,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['LogisticRegression'] = evaluateBinaryClassification(y_predict_LR,y_test)
  # print(compare_df)

avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['Logistic Regression'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

               # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  nb = GaussianNB()
  nb.fit(X_train, y_train)

  y_predict_NB = nb.predict(X_test)
  # print(y_predict_NB)

  list = evaluateBinaryClassification(y_predict_NB,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['NaiveBayes'] = evaluateBinaryClassification(y_predict_NB,y_test)
  # print(compare_df)

avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['Naive Bayes'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

Support Vector Machine

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

               # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  model = svm.SVC()
  model.fit(X_train, y_train)
  y_predict_SVM = model.predict(X_test)
  # print(y_predict_SVM)
  
  list = evaluateBinaryClassification(y_predict_SVM,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['SVM'] = evaluateBinaryClassification(y_predict_SVM,y_test)
  # print(compare_df)

avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['SVM'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

Random Forest Classifier

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

               # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)


  RandomForest = RandomForestClassifier()
  RandomForest.fit(X_train,y_train)

  y_predict_RR = RandomForest.predict(X_test)
  # print(y_predict_RR)

  list = evaluateBinaryClassification(y_predict_RR,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['RandomForest'] = evaluateBinaryClassification(y_predict_RR,y_test)
  # print(compare_df)

  # print("<___________________________>")
  # print(RandomForest.oob_score_)

  # print("<___________________________>")

avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['Random Forest'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

               # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  lda = LinearDiscriminantAnalysis()          #not working for n_components=2
  X_train = lda.fit_transform(X_train, y_train)
  X_test = lda.transform(X_test)

  classifier = RandomForestClassifier()
  classifier.fit(X_train, y_train)

  #plotDecisionTree(classTree, feature_names=X_train.columns, class_names=classTree.classes_)
  y_predict_LDA = classifier.predict(X_test)
  # print(y_predict_LDA)

  list = evaluateBinaryClassification(y_predict_LDA,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['LDA'] = evaluateBinaryClassification(y_predict_LDA,y_test)
  # print(compare_df)

avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['LDA'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

QDA Extra Trees Classifier

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier

X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

# Building the model
extra_tree_forest = ExtraTreesClassifier()
  
# Training the model
extra_tree_forest.fit(X, Y)
  
# Computing the importance of each feature
feature_importance = extra_tree_forest.feature_importances_
  
# Normalizing the individual importances
feature_importance_normalized = np.std([tree.feature_importances_ for tree in extra_tree_forest.estimators_],axis = 0)

# Plotting a Bar Graph to compare the models
plt.bar(X.columns, feature_importance_normalized)
plt.tick_params(axis='x', which='major', labelsize=6)
plt.xticks(rotation = 90)
plt.xlabel('Feature Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of different Feature Importances')
plt.show()


Gradient Boosting Classifier

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

               # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  model = GradientBoostingClassifier()
  model.fit(X_train, y_train)

  y_predict_GradientBoostingClassifier = model.predict(X_test)
  # print(y_predict_GradientBoostingClassifier)

  list = evaluateBinaryClassification(y_predict_GradientBoostingClassifier,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['GradientBoostingClassifier'] = evaluateBinaryClassification(y_predict_GradientBoostingClassifier,y_test)
  # print(compare_df)

avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['Gradient Boosting'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

LGMB Classifier

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

               # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  model = LGBMClassifier()
  model.fit(X_train, y_train)
  y_predict_LGMBclassifier = model.predict(X_test)
  # print(y_predict_LGMBclassifier)

  list = evaluateBinaryClassification(y_predict_LGMBclassifier,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['LGBMclassifier'] = evaluateBinaryClassification(y_predict_LGMBclassifier,y_test)
  # print(compare_df)

avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['LGBMclassifier'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

XgBoost Classifier

In [ ]:
# NOT WORKING

# kfold = KFold(n_splits=10, shuffle=True)
# avg_accuracy = 0
# avg_recall = 0
# avg_precision = 0
# avg_fscore = 0
# max_accuracy = 0;
# for train_index,test_index in kfold.split(data):

#                # Get the train-test subsets
#   X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
#   y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]
#         # Scale the X features according to training data 
#   scaler = StandardScaler()
#   X_train = scaler.fit_transform(X_train)
#   X_test = scaler.transform(X_test)

#   model = XGBClassifier()
#   model.fit(X_train, y_train)
#   y_predict_XgBoost = model.predict(X_test)
#   # print(y_predict_XgBoost)

#   list = evaluateBinaryClassification(y_predict_XgBoost,y_test)
#   max_accuracy = max(max_accuracy,list[0])
#   avg_accuracy += list[0]
#   avg_recall += list[1]
#   avg_precision += list[2]
#   avg_fscore += list[3]

#   # compare_df.loc['XgBoost'] = evaluateBinaryClassification(y_predict_XgBoost,y_test)
#   # print(compare_df)

# avg_accuracy /= kfold.get_n_splits()
# avg_recall /= kfold.get_n_splits()
# avg_precision /= kfold.get_n_splits()
# avg_fscore /= kfold.get_n_splits()
# compare_df.loc['XgBoost'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# # print(compare_df)

MLP Classifier

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for train_index,test_index in kfold.split(data):

               # Get the train-test subsets
  X_train, X_test = data.iloc[train_index,:-1], data.iloc[test_index,:-1]
  y_train, y_test = data.iloc[train_index,-1], data.iloc[test_index,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)


  mlp = MLPClassifier()
  mlp.fit(X_train, y_train)
  y_predict_mlp = mlp.predict(X_test)

  list = evaluateBinaryClassification(y_predict_mlp,y_test)
  max_accuracy = max(max_accuracy,list[0])
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  #compare_df = pd.DataFrame()
  # compare_df.loc['MLP'] = evaluateBinaryClassification(y_predict_mlp,y_test)
  # print(compare_df)

avg_accuracy /= kfold.get_n_splits()
avg_recall /= kfold.get_n_splits()
avg_precision /= kfold.get_n_splits()
avg_fscore /= kfold.get_n_splits()
compare_df.loc['MLP'] = [max_accuracy,avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

In [ ]:
print(compare_df)